# 환경변수 로딩

In [22]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv(override=True)
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
# PINECONE_API_KEY

# pinecone 클라이언트 객체, index 객체 생성

In [23]:
from pinecone import Pinecone
from pinecone import Pinecone, ServerlessSpec

#pinecone 객체생성
pc = Pinecone(api_key=PINECONE_API_KEY)


## 인덱스가 존재하면 삭제

In [24]:
index_name = "wiki"

# Pinecone에 있는 모든 인덱스를 순회합니다.
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

## 인덱스 생성

In [25]:
# Pinecone 인덱스를 생성합니다.
# 인덱스 이름은 "wiki"이고, 차원은 1536, 메트릭은 코사인 유사도를 사용합니다.
# 인덱스는 AWS의 us-east-1 리전에서 서버리스 사양으로 생성됩니다.
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원, openapi embeding model을 사용함. 정확하게 일치
    metric="cosine",  # 모델 메트릭, openapi embeding model 에서 사용하는 것 확인
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-r8d3erp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [26]:
# 인덱스 객체 생성
index = pc.Index(index_name)

In [27]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# vector db에 데이터 인덱싱

## 데이터 수집, 데이터 로드
- 허깅페이스 오픈 소스 데이터 사용
- pip install datasets

In [28]:
from datasets import load_dataset

In [29]:
#  https://huggingface.co/datasets/legacy-datasets/wikipedia
dataset = load_dataset(
    "wikimedia/wikipedia",
    "20231101.ko",   # simple 영어 위키
    split="train[:100]"
)

In [30]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [31]:
# 데이터 확인
for row in dataset:
    print(row)

{'id': '5', 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터', 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 

## 데이터 분할객체 생성

In [32]:
# 데이터 분할 객체 생성
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,  # 한 조각(chunk)의 최대 길이
    chunk_overlap = 30,   # 이전 청크와 겹치는 글자 수, 문맥 연결성 유지
    length_function = len,  # 글자 수 기준 # chunk_size 단위
    separators=["\n\n", "\n", " ", ""]   # 텍스트를 나눌 때 우선적으로 사용하는 기준
)

## 임베딩 객체 생성

In [33]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key = OPENAI_API_KEY
)

# 레코드 업로딩(upsert)

## 데이터 분할 테스트

In [34]:
from tqdm.auto import tqdm
for i, doc in enumerate(tqdm(dataset)):
    full_text = doc["text"]
    # print(i, " : ", full_text[:50])
    # print("-"*30)

    # chunk 단위로 분할(하나의 문서를 여러개의 chunk로 분할)
    chunks = splitter.split_text(full_text)
    print(f"{i} : {chunks}")
    print("-"*30)
    print("chunk 갯수", len(chunks))

  0%|          | 0/100 [00:00<?, ?it/s]

0 : ['제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', '정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.', '외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였다.', '그는 주이란 미국 대사관 인질 사건의 인질 구

## 메타데이터 구성

In [37]:
from pprint import pformat
metadata_base = {
    'wiki_id': str(doc["id"]),  # wikipedia 문서 id
    'url': doc["url"],  # wikipedia 문서 url
    'title': doc["title"]  # wikipedia 문서 title
}

record = {
    'chunk_id': 1,
    **metadata_base,  # 언패킹
    # 'full_text': full_text
}

print(pformat(record))

{'chunk_id': 1,
 'title': '중국의 역사',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A4%91%EA%B5%AD%EC%9D%98%20%EC%97%AD%EC%82%AC',
 'wiki_id': '246'}


## DB에 저장

In [38]:
from tqdm.auto import tqdm
from uuid import uuid4
import time

# 업로드 설정
BATCH_SIZE = 100  # 100개씩 Vector DB에 저장
NAMESPACE = "wiki_ns1"
texts = []        # chunk 텍스트 모음
metadatas = []    # 메타데이터 모음

# 문서 하나씩 처리하기
# dataset : Wikipedia 문서 목록
for doc in tqdm(dataset):
    # full_text : 하나의 위키 문서 전체 내용
    full_text = doc["text"]

    # 문서 정보(메타데이터) 준비
    chunk_metadata = {
        "wiki_id": str(doc["id"]),
        "title": doc["title"],
        "url": doc["url"]
    }

    # 문서를 chunk로 나누기(너무 긴 문서는 LLM이 처리 못함, 작은 조각(chunk)으로 나눔)
    chunks = splitter.split_text(full_text)

    for idx, chunk in enumerate(chunks):
        # texts / metadatas에 임시 저장
        texts.append(chunk)
        metadatas.append({
            "chunk_id": idx,   # chunk_id : 같은 문서 내 몇 번째 조각인지
            # "full_text": chunk[:100], 
            "chunk_text": chunk, 
            **chunk_metadata
        })
        # 100개 모이면 Vector DB에 저장
        if len(texts) == BATCH_SIZE:
            ids = [str(uuid4()) for _ in range(BATCH_SIZE)]
        
            # 임베딩 -> Pinecone 업로드
            embeddings = embedding.embed_documents(texts)

            index.upsert(
                vectors=zip(ids, embeddings, metadatas),
                namespace=NAMESPACE
            )
            # 초기화 후 다음 배치로 
            texts = []
            metadatas = []
            time.sleep(1)

# 마지막 남은 chunk 업로드 누락
if texts:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeddings = embedding.embed_documents(texts)
    index.upsert(
        vectors=zip(ids, embeddings, metadatas),
        namespace=NAMESPACE
    )

  0%|          | 0/100 [00:00<?, ?it/s]